# Lernen im mehrschichtigen Netz
## Entwurf des Netzes
Das folgende Netz soll programmiert werden. Es besitzt zwei Eingabeneuronen plus ein Bias-Neuron in der Eingabeschicht, drei Neuronen in der verdeckten Schicht und zwei Neuronen für die Ausgabe. Die Bezeichnungen für die Gewichte sind so, wie sie im folgenden benutzt werden, die Indizes geben an "woher" "wohin".
![Netzaufbau](bilder/NeuronNetz332.png "Titel")
Mit diesem Netz soll die XOR-Funktion berechnet und gelernt werden.
Für die folgenden Beispielrechnungen werden die folgenden Vereinfachungen gemacht:
* alle Gewichte werden bis auf zwei Ausnahmen  mit 0,1 initialisiert
* der Lernfaktor beträgt ebenfalls 1


## Berechnung der Netzeingabe
Als Eingabe soll der Vektor (1, 0, 1) dienen, die erste 1 ist vom Bias-Neuron, die beiden anderen repräsentieren x<sub>1</sub> und x<sub>2</sub>.
Für den Knoten 1 der versteckten Schicht berechnet sich die Eingabe wie folgt:
<span class="math">$$HI_1 =  x_0 \cdot w^{IH}_{01} + x_1 \cdot w^{IH}_{11} + x_2 \cdot w^{IH}_{21} = 0,3 $$</span> 
Das lässt sich sehr elegant als Produkt der Gewichtsmatrix mit dem Eingabevektor beschreiben und implementieren und ergibt HI, die Eingabe in die versteckte Schicht:
<span class="math">$$ IN\cdot W^{IH} = HI  $$</span>

<math>$$\begin{pmatrix}  1  & 0 & 1  \end{pmatrix} \cdot \begin{pmatrix} 0,1 & 0,2 & 0,1 \\ 0,1 & 0,1 & 0,1 \\ 0,1 & 0,1 & 0,1\end{pmatrix}  
      =  \begin{pmatrix}0,2 & 0,3 & 0,2 \end{pmatrix} $$</math>  
    
Dabei kann die erste Komponente des Ergebnisvektors als Eingabe in den Bias-Knoten der verdeckten Schicht interpretiert werden und wird ignoriert. In der Implementierung haben die Matrizen aus Gründen der Einheitlichkeit stets diese Dimensionen und auch dabei werden einzelne Ergebnisse ignoriert.



## Berechnung der Aktivierung und der Ausgabe
Auf die Eingaben der verdeckten Schicht wird nun jeweils die Stufenfunktion angewendet und man erhält HO, die Ausgabe der versteckten Schicht:
<math>$$ HO = \sigma(HI) = \sigma\begin{pmatrix}  0,2  & 0,3  & 0,2  \end{pmatrix} =  \begin{pmatrix}  0,5498 & 0,5744 &  0,5498 \end{pmatrix} $$</math>

Mit der Ausgabe der versteckten Schicht berechnet man die Eingabe in das Ausgabeneuron FI (final in).

$$ HO \cdot W^{HO}= FI  $$

$$\begin{pmatrix}  0,5498 & 0,5744 &  0,5498 \end{pmatrix} \cdot 
     \begin{pmatrix}  0,2 & 0,1  \\ 0,1 & 0,1  \\ 0,1 & 0,1  \end{pmatrix} =  \begin{pmatrix}0,2224 & 0,1674 \end{pmatrix} $$ 

Auch auf diese Eingabe wird wieder die Sigmoid-Funktion angewendet, man erhält FO (final out):

$$ FO = \sigma(FI) \begin{pmatrix}0,2224 & 0,1674 \end{pmatrix} = \begin{pmatrix}0,5554 & 0,5418 \end{pmatrix}  $$

Damit hat die Eingabe einmal das Netz durchlaufen und verursacht einen Fehler, der dann zur Korrektur der Gewichte benutzt wird.
$$ E = \frac {1}{2} (t- FO)^2$$

## Backpropagation
### Gewichtskorrektur aus der Outputschicht heraus
Wir nehmen an, dass unser Netz in der Ausgabeschicht einen Fehler E gemacht hat, der sich aus der Differenz des gewünschten Wertes t und der berechneten Ausgabe FO ergibt. 

$$ E = \frac {1}{2} (t- FO)^2$$

und zur Erinnerung setzt sich FO zusammen aus:

$$FO = \sigma(FI)= \sigma(HO \cdot W^{HO}) $$

Um den Einfluss der einzelnen Gewichte auf den Fehler zu bestimmen und diese entsprechend zu korrigieren, wird der Fehler nach dem jeweiligen Gewicht abgeleitet. Damit erhält man den Gradienten des Fehlers, das ist ein Vektor, der die Richtung der größten Steigung der Fehlerfunktion angibt. Die Korrektur muss dann also entgegengerichtet zu diesem Vektor erfolgen. 
Allgemein gilt:
$$\Delta W^{HO}= -\eta \frac {\delta E}{\delta w^{HO}}$$

Für die Ableitung muss nun mehrfach die Kettenregel angewendet werden.
Für eine Komponente dieses Vektors soll diese Ableitung berechnet werden:
$$\frac {\delta E}{\delta w^{HO}_{ij}}= \frac {\delta(\frac {1}{2} (t- FO)^2)}{\delta w^{HO}_{ij}} $$

mit der Kettenregel ergibt sich

$$\frac {\delta E}{\delta w^{HO}_{ij}}=-(t-FO) \cdot  \frac {\delta FO}{\delta w^{HO}_{ij}} $$

mit $$FO = \sigma(FI)$$ und 

$$ \sigma '(FI) = \sigma(FI)\cdot (1-\sigma(FI))$$

für die Sigmoid-Funktion bekommt man

$$\frac {\delta E}{\delta w^{HO}_{ij}}=-(t-FO) \cdot  \frac {\delta (\sigma(FI))}{\delta w^{HO}_{ij}} $$

$$\frac {\delta E}{\delta w^{HO}_{ij}}=-(t-FO) \cdot \sigma(FI)\cdot (1- \sigma(FI))\cdot \frac {\delta (FI)}{\delta w^{HO}_{ij}} $$

oder 

$$\frac {\delta E}{\delta w^{HO}_{ij}}=-(t-FO) \cdot FO\cdot (1- FO)\cdot \frac {\delta (FI)}{\delta w^{HO}_{ij}} $$

Die Eingabe in die Outputschicht FI schließlich hängt direkt von den Gewichten ab, in unserem Beispiel gilt

$$FI = w^{HO}_{10}\cdot HO_0 + w^{HO}_{11}\cdot HO_1 +w^{HO}_{12}\cdot HO_2 $$

In die Ableitung geht damit nur der Summand ein, in dem das Gewicht, nach dem abgeleitet wird, steht. 

$$\frac {\delta E}{\delta w^{HO}_{10}}=-(t-FO) \cdot FO\cdot (1- FO)\cdot \frac {\delta (FI)}{\delta w^{HO}_{10}} $$

$$\frac {\delta E}{\delta w^{HO}_{10}}=-(t-FO) \cdot FO\cdot (1- FO)\cdot HO_0 $$

Für die Gewichtskorrektur wird noch mit (-1) und dem Lernfaktor multipliziert. 

$$\Delta w^{HO}_{10}= \eta \cdot (t-FO) \cdot FO\cdot (1- FO)\cdot HO_0 \cdot $$

### Gewichtskorrektur aus der Hidden-Schicht heraus  muss neu gemacht werden

Analog wie oben geht man vor um die Korrektur von w^{IH} zu berechnen. Auch hier muss der Anteil der einzelnen Gewichte am Fehler bestimmt werden. Da man davon ausgeht, dass jeder der Knoten der Hidden-Schicht entsprechend des Gewichts w^{HO} zum Gesamtfehler beigetragen hat, wichtet man den Fehler dieser Knoten mit diesen Gewichten. Es gilt für den i-ten Knoten der Hidden-Schicht

$$ delta^{HO}_i = der_i \cdot w^{HO}_{1i} \cdot delta^{FO}$$

Da es hier insgesamt 9 Gewichtskorrekturen gibt, schreibt sich das besser als Matrizenmultiplikation:

$$\Delta w^{IH}_{ji}= \eta \cdot \begin{pmatrix} -0,0667 \\ -0,0131 \\ -0,0131 \end{pmatrix} \cdot 
     \begin{pmatrix}  1  \\ 0  \\ 0  \end{pmatrix}^T 
     = \eta \cdot \begin{pmatrix} -0,0667 \\ -0,0131 \\ -0,0131 \end{pmatrix} \cdot 
     \begin{pmatrix}  1  & 0  & 0  \end{pmatrix}
     = \eta \cdot \begin{pmatrix}-0,0667 & 0 & 0\\ -0,0131 & 0 & 0 \\-0,0131 &0 &0\end{pmatrix} $$ 

Allgemein gilt also:

$$\Delta w^{IH}_{ji}= \eta \cdot delta^{HO}_j  \cdot  IN_i^T  $$

Rechenbeispiel als Excel-Tabelle:
hier gelten:
$$der = FO\cdot (1- FO) $$und für die Output-Schicht:

$$delta = (t-FO)\cdot der$$
sowie 
$$\Delta w^{HO}_{10}= \eta \cdot delta \cdot HO_0$$
![Berechnung](bilder/ergebnisseXOR101.png "Rechnung")